In [1]:
#Import packages needed
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import requests
import gmaps
from config import gkey, pkey
from pprint import pprint
gmaps.configure(api_key=gkey)

## Bringing in the data

In [2]:
#Bringing in cime data and making it a dataframe
crimes_csv = "LACrimes2013Zip.csv"
crimes_df = pd.read_csv(crimes_csv, nrows=1000) #For speed of example we have used only 1,000 rows
crimes_df.head()

,Date.Rptd,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Status,Status.Desc,LOCATION,Cross.Street,Location.1,Formatted Address,Zipcode
0,03/20/2013,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,UNK,Unknown,OXFORD,OAKWOOD,"(34.0776, -118.308)","4650-4652 Oakwood Ave, Los Angeles, CA 90004, USA",90004
1,03/10/2013,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,UNK,Unknown,ODIN ST,CAHUENGA BL,"(34.1113, -118.3336)","2314 N Cahuenga Blvd, Los Angeles, CA 90068, USA",90068
2,12/18/2013,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,UNK,Unknown,105TH ST,CROESUS AV,"(33.9406, -118.2338)","2148 E 105th St, Los Angeles, CA 90002, USA",90002
3,10/18/2013,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,UNK,Unknown,101ST ST,JUNIPER ST,"(33.9449, -118.2332)","10100-10198 Juniper St, Los Angeles, CA 90002,...",90002
4,05/26/2013,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),UNK,Unknown,1300 W SEPULVEDA BL,NaN,"(33.8135, -118.2992)","1303 Sepulveda Blvd, Torrance, CA 90501, USA",90501


In [3]:
#Bringing in bike data and making it a dataframe
bike_csv = "los-angeles-metro-bike-share-trip-data/metro-bike-share-trip-data.csv"
bike_df = pd.read_csv(bike_csv, nrows=1000) #For speed of example we have used only 1,000 rows
bike_df.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Trip Route Category,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts
0,18222186,15060,2017-01-19T17:05:00.000,2017-01-19T21:16:00.000,3031.0,34.044701,-118.252441,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9518671,77160,2016-10-09T14:37:00.000,2016-10-10T12:03:00.000,NaN,NaN,NaN,3000.0,NaN,NaN,...,One Way,Monthly Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20444932,86400,2017-02-18T10:15:00.000,2017-02-20T15:20:00.000,3026.0,34.063179,-118.245880,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20905031,18840,2017-02-27T12:26:00.000,2017-02-27T17:40:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21031476,86400,2017-02-27T20:26:00.000,2017-03-01T09:49:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,One Way,Flex Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Looking at and cleaning the data

In [4]:
# Look at the size of the data
crimes_df.shape

(1000, 16)

In [5]:
# Look at the size of the data
bike_df.shape

(1000, 22)

In [6]:
# Check to see if any NaN's in crimes and how much
crimes_df["Zipcode"].isnull().sum()

0

In [7]:
# At this point we have recognized that 'Zip Codes' in the bike dataframe are missing or wrong 
# so we have decided to get zip codes by using geolocation and google api

In [8]:
# Check to see if any NaN's in crimes and how much
bike_df["Starting Station Latitude"].isnull().sum()

12

In [9]:
# Check to see if any NaN's in crimes and how much
bike_df["Starting Station Longitude"].isnull().sum()

12

In [10]:
# Drop the NaN's in bikes and check to see how much data is left
bike_df_no_nans = bike_df.dropna(subset=["Starting Station Latitude", "Starting Station Longitude"])
bike_df_no_nans.shape

(988, 22)

In [11]:
bike_df_no_nans.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Trip Route Category,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts
0,18222186,15060,2017-01-19T17:05:00.000,2017-01-19T21:16:00.000,3031.0,34.044701,-118.252441,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20444932,86400,2017-02-18T10:15:00.000,2017-02-20T15:20:00.000,3026.0,34.063179,-118.245880,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20905031,18840,2017-02-27T12:26:00.000,2017-02-27T17:40:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21031476,86400,2017-02-27T20:26:00.000,2017-03-01T09:49:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,One Way,Flex Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,21784648,14760,2017-03-10T17:38:00.000,2017-03-10T21:44:00.000,3065.0,34.060558,-118.238327,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Finding new Zip Codes

In [13]:
# Quick look at the Zip Codes and we can tell they are wrong. LA has zip codes in the 90000's
# Create new Zip Code Column
bike_df_no_nans.assign(New_Zip = "")
bike_df_no_nans.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Trip Route Category,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts
0,18222186,15060,2017-01-19T17:05:00.000,2017-01-19T21:16:00.000,3031.0,34.044701,-118.252441,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20444932,86400,2017-02-18T10:15:00.000,2017-02-20T15:20:00.000,3026.0,34.063179,-118.245880,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20905031,18840,2017-02-27T12:26:00.000,2017-02-27T17:40:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21031476,86400,2017-02-27T20:26:00.000,2017-03-01T09:49:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,One Way,Flex Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,21784648,14760,2017-03-10T17:38:00.000,2017-03-10T21:44:00.000,3065.0,34.060558,-118.238327,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
count = 0
for index, row in bike_df_no_nans.iterrows():
    
    lat = row["Starting Station Latitude"]
    long = row["Starting Station Longitude"]


    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
                  'latlng={lat},{long}&key={gkey}').format(lat=lat, long=long, gkey=gkey)
    response = requests.get(target_url).json()
    
    try:
        bike_df_no_nans.loc[index, 'New Zip'] = int(response['results'][0]['address_components'][7]['long_name'])
#         print(f"Adding {response['results'][0]['address_components'][7]['long_name']}")
    except:
        bike_df_no_nans.loc[index, 'New Zip'] = np.nan
        count += 1
print(f'Finished retrieving zip code data. Couldn\'t find {count} zip codes.')
bike_df_no_nans.head()

/Users/kevinheaney/anaconda3/envs/WashU/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/kevinheaney/anaconda3/envs/WashU/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Finished retrieving zip code data. Couldn't find 153 zip codes.


,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts,New Zip
0,18222186,15060,2017-01-19T17:05:00.000,2017-01-19T21:16:00.000,3031.0,34.044701,-118.252441,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90014.0
2,20444932,86400,2017-02-18T10:15:00.000,2017-02-20T15:20:00.000,3026.0,34.063179,-118.245880,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0
3,20905031,18840,2017-02-27T12:26:00.000,2017-02-27T17:40:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0
4,21031476,86400,2017-02-27T20:26:00.000,2017-03-01T09:49:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,Flex Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,21784648,14760,2017-03-10T17:38:00.000,2017-03-10T21:44:00.000,3065.0,34.060558,-118.238327,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0


In [15]:
# See how many 'New Zip' NaN's there are
bike_df_no_nans["New Zip"].isnull().sum()

153

In [16]:
bike_df_no_nans.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts,New Zip
0,18222186,15060,2017-01-19T17:05:00.000,2017-01-19T21:16:00.000,3031.0,34.044701,-118.252441,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90014.0
2,20444932,86400,2017-02-18T10:15:00.000,2017-02-20T15:20:00.000,3026.0,34.063179,-118.245880,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0
3,20905031,18840,2017-02-27T12:26:00.000,2017-02-27T17:40:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0
4,21031476,86400,2017-02-27T20:26:00.000,2017-03-01T09:49:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,Flex Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,21784648,14760,2017-03-10T17:38:00.000,2017-03-10T21:44:00.000,3065.0,34.060558,-118.238327,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0


In [17]:
# Drop the NaN's from dataframe
bike_df_no_nans_zip = bike_df_no_nans.dropna(subset=["New Zip"])
bike_df_no_nans_zip.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts,New Zip
0,18222186,15060,2017-01-19T17:05:00.000,2017-01-19T21:16:00.000,3031.0,34.044701,-118.252441,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90014.0
2,20444932,86400,2017-02-18T10:15:00.000,2017-02-20T15:20:00.000,3026.0,34.063179,-118.245880,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0
3,20905031,18840,2017-02-27T12:26:00.000,2017-02-27T17:40:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0
6,21784648,14760,2017-03-10T17:38:00.000,2017-03-10T21:44:00.000,3065.0,34.060558,-118.238327,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90012.0
7,22506739,79020,2017-03-17T19:20:00.000,2017-03-18T17:17:00.000,3063.0,34.049198,-118.252831,3000.0,NaN,NaN,...,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90013.0


## Get the clean dataframe to be used

In [18]:
final_bike = bike_df_no_nans_zip[["New Zip", "Trip ID"]].copy()
final_bike.head()

,New Zip,Trip ID
0,90014.0,18222186
2,90012.0,20444932
3,90012.0,20905031
6,90012.0,21784648
7,90013.0,22506739


In [19]:
final_bike.shape

(835, 2)

In [20]:
groupby_zip_bikes = final_bike.groupby("New Zip").count()
groupby_zip_bikes

,Trip ID
New Zip,
90007.0,2
90012.0,276
90013.0,205
90014.0,134
90015.0,96
90017.0,74
90021.0,35
90071.0,6
90079.0,3


In [21]:
groupby_zip_bikes.reset_index(inplace=True)
groupby_zip_bikes

,New Zip,Trip ID
0,90007.0,2
1,90012.0,276
2,90013.0,205
3,90014.0,134
4,90015.0,96
5,90017.0,74
6,90021.0,35
7,90071.0,6
8,90079.0,3
9,90291.0,4


In [22]:
groupby_zip_bikes.rename(index=str, columns={"New Zip":"new_zip","Trip ID": "bike_ride_count"}, inplace=True)
groupby_zip_bikes

,new_zip,bike_ride_count
0,90007.0,2
1,90012.0,276
2,90013.0,205
3,90014.0,134
4,90015.0,96
5,90017.0,74
6,90021.0,35
7,90071.0,6
8,90079.0,3
9,90291.0,4


In [23]:
# Get only the columns that we need and make a new dataframe
new_crimes_df = crimes_df[["DR.NO", "Zipcode"]].copy()
new_crimes_df.head()

,DR.NO,Zipcode
0,132007717,90004
1,130608787,90068
2,131820260,90002
3,131817514,90002
4,130510483,90501


In [24]:
new_crimes_df.shape

(1000, 2)

In [25]:
groupby_zip_crimes = new_crimes_df.groupby("Zipcode").count()
groupby_zip_crimes.head()

,DR.NO
Zipcode,
90001,3
90002,14
90003,30
90004,7
90005,6


In [26]:
# Reset set the index
groupby_zip_crimes.reset_index(inplace=True)
groupby_zip_crimes.head()

,Zipcode,DR.NO
0,90001,3
1,90002,14
2,90003,30
3,90004,7
4,90005,6


In [27]:
groupby_zip_crimes.rename(index=str, columns={"Zipcode":"zipcode","DR.NO": "crime_count"}, inplace=True)
groupby_zip_crimes.head()

,zipcode,crime_count
0,90001,3
1,90002,14
2,90003,30
3,90004,7
4,90005,6


## Connect to local database

In [28]:
rds_connection_string = f"postgres:{pkey}@localhost:5432/bike_crime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

## Check for tables

In [29]:
engine.table_names()

['crimes_la', 'bike_la']

In [30]:
groupby_zip_bikes.to_sql(name='bike_la', con=engine, if_exists='append', index=False)

In [31]:
groupby_zip_crimes.to_sql(name='crimes_la', con=engine, if_exists='append', index=False)

In [32]:
pd.read_sql_query('SELECT * FROM bike_la', con=engine).head()

,new_zip,bike_ride_count
0,90012,26
1,90013,23
2,90014,12
3,90015,10
4,90017,6


In [33]:
pd.read_sql_query('SELECT * FROM crimes_la', con=engine).head()

,zipcode,crime_count
0,90002,2
1,90004,2
2,90007,1
3,90008,3
4,90012,1
